In [1]:
import pandas as pd
import altair as alt

In [2]:
cols = ["IdLandkreis", "Meldedatum", "AnzahlFall", "NeuerFall"]
covid = pd.read_csv('https://npgeo-corona-npgeo-de.hub.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0.csv', usecols=cols, parse_dates=['Meldedatum'])
einwohner = pd.read_csv('../Einwohnerzahlen.csv')

In [ ]:
covid.Meldedatum = pd.DatetimeIndex(covid.Meldedatum.dt.date)
covid.tail()

In [5]:
covid.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1799157 entries, 0 to 1799156
Data columns (total 4 columns):
 #   Column       Dtype              
---  ------       -----              
 0   AnzahlFall   int64              
 1   Meldedatum   datetime64[ns, UTC]
 2   IdLandkreis  int64              
 3   NeuerFall    int64              
dtypes: datetime64[ns, UTC](1), int64(3)
memory usage: 54.9 MB


In [ ]:
key_data = covid[covid.NeuerFall != -1].groupby(['IdLandkreis', 'Meldedatum']).sum()
key_data

In [ ]:
idx = pd.DatetimeIndex(pd.date_range(start=covid.Meldedatum.min(), end=covid.Meldedatum.max(), freq='1D'))
multi_idx = pd.MultiIndex.from_product([key_data.index.levels[0], idx], names=["IdLandkreis", "Meldedatum"])

key_data = key_data.reindex(multi_idx, fill_value=0)
key_data

In [ ]:
cases_7d = []
for group, data in key_data.groupby(level=0):
    cases_7d.extend(data.reset_index().set_index('Meldedatum').rolling(window="7D").sum()['AnzahlFall'].to_list())
key_data['AnzahlFall7T'] = cases_7d

In [ ]:
key_data.reset_index(inplace=True)

In [ ]:
key_data = key_data.merge(einwohner, left_on="IdLandkreis", right_on="AdmUnitId")
key_data['Inz7T'] = key_data.AnzahlFall7T / key_data.EWZ * 100000

In [ ]:
alt.Chart(key_data[key_data.reset_index().IdLandkreis.isin([12063, 12064, 3251])]).mark_line().encode(
    x='Meldedatum',
    y=alt.Y('Inz7T'),
    color="IdLandkreis:N",
).interactive(True).properties(width=800)